In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q /content/drive/MyDrive/SB/Assignment2/train_data.zip -d /content/drive/MyDrive/SB/Assignment2/
!unzip -q /content/drive/MyDrive/SB/Assignment2/test_data.zip -d /content/drive/MyDrive/SB/Assignment2/
!unzip -q /content/drive/MyDrive/SB/Assignment2/val_data.zip -d /content/drive/MyDrive/SB/Assignment2/

In [4]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128, 64)),
        transforms.ToTensor(),
        # transforms.Grayscale(num_output_channels=3),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# image_train = datasets.ImageFolder("/content/drive/MyDrive/SB/Assignment2/train_data", data_transforms['train'])
# image_val = datasets.ImageFolder("/content/drive/MyDrive/SB/Assignment2/val_data", data_transforms['train'])

# train_data = torch.utils.data.DataLoader(image_train, batch_size=4, shuffle=True, num_workers=4, transforms=)
# val_data = torch.utils.data.DataLoader(image_val, batch_size=4, shuffle=True, num_workers=4)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


data_dir = '/content/drive/MyDrive/SB/Assignment2/'
image_datasets = {x: datasets.ImageFolder(data_dir+x+"_data",
                                          data_transforms['train'])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [20]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 100)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [22]:
# use this if u want to use Alexnet

model_ft = models.alexnet(pretrained=True)
num_ftrs = model_ft.classifier[6].in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 100)
model_ft.classifier[6] = nn.Linear(num_ftrs, 100)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.5091 Acc: 0.0340
val Loss: 4.5660 Acc: 0.0308

Epoch 1/24
----------
train Loss: 3.7167 Acc: 0.1434
val Loss: 4.3357 Acc: 0.0843

Epoch 2/24
----------
train Loss: 2.7961 Acc: 0.3174
val Loss: 4.4967 Acc: 0.0950

Epoch 3/24
----------
train Loss: 2.0715 Acc: 0.4693
val Loss: 4.3722 Acc: 0.1539

Epoch 4/24
----------
train Loss: 1.5840 Acc: 0.5822
val Loss: 4.6568 Acc: 0.1372

Epoch 5/24
----------
train Loss: 1.3051 Acc: 0.6494
val Loss: 5.4869 Acc: 0.1258

Epoch 6/24
----------
train Loss: 1.1083 Acc: 0.7138
val Loss: 4.7016 Acc: 0.1426

Epoch 7/24
----------
train Loss: 0.3937 Acc: 0.8961
val Loss: 4.9784 Acc: 0.2095

Epoch 8/24
----------
train Loss: 0.1368 Acc: 0.9649
val Loss: 5.4189 Acc: 0.2095

Epoch 9/24
----------
train Loss: 0.0811 Acc: 0.9805
val Loss: 5.7581 Acc: 0.2115

Epoch 10/24
----------
train Loss: 0.0552 Acc: 0.9893
val Loss: 6.0669 Acc: 0.2068

Epoch 11/24
----------
train Loss: 0.0405 Acc: 0.9901
val Loss: 6.3125 Acc: 0.2095

Ep

In [11]:
!rm -R /content/drive/MyDrive/SB/Assignment2/test_data/.ipynb_checkpoints

rm: cannot remove '/content/drive/MyDrive/SB/Assignment2/test_data/.ipynb_checkpoints': No such file or directory


In [12]:
torch.save(model_ft.state_dict(), "/content/drive/MyDrive/SB/Assignment2/resnet50_noG.pth")

In [24]:
test_dataset = datasets.ImageFolder("/content/drive/MyDrive/SB/Assignment2/test_data", data_transforms['train'])
datattest = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=2)

In [27]:
# evalvation - calculate test accuracy
with torch.no_grad():
    accuracy = 0
    correct = 0
    total = 0

    for data in datattest:
        images, new_labels = data
        images = images.to(device)
        new_labels = new_labels.to(device)
        outputs = model_ft(images)
        _ , pred = torch.max(outputs, 1)
        total += new_labels.size(0)
        correct += torch.sum(pred == new_labels.data)

    accuracy = correct.double() / total

print('Test accuracy is: ' + str(accuracy.item()))

Test accuracy is: 0.11224489795918367


In [84]:
model_ft.load_state_dict(torch.load("/content/drive/MyDrive/SB/Assignment2/resnet50.pth"))

<All keys matched successfully>